In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("topical_chat.csv")
df

,conversation_id,message,sentiment
0,1,Are you a fan of Google or Microsoft?,Curious to dive deeper
1,1,Both are excellent technology they are helpfu...,Curious to dive deeper
2,1,"I'm not a huge fan of Google, but I use it a...",Curious to dive deeper
3,1,Google provides online related services and p...,Curious to dive deeper
4,1,"Yeah, their services are good. I'm just not a...",Curious to dive deeper
...,...,...,...
188373,8628,"Wow, it does not seem like that long. Since I...",Surprised
188374,8628,"I havent seen that episode, I might google it...",Curious to dive deeper
188375,8628,I don't think I have either. That's an insane...,Curious to dive deeper
188376,8628,"I did, my little brother used to love Thomas ...",Happy


In [17]:
#Assign Numeric Values of 1 or 0 to Categorical Data: Happy, Neutral, Sad, Surprised, Curious to dive deeper (shortened to curious)
df['Curious'] = np.where(df['sentiment'].str.contains("Curious"), 1, 0)
df['Happy'] = np.where(df['sentiment'].str.contains("Happy"), 1, 0)
df['Sad'] = np.where(df['sentiment'].str.contains("Sad"), 1, 0)
df['Neutral'] = np.where(df['sentiment'].str.contains("Neutral"), 1, 0)
df['Surprised'] = np.where(df['sentiment'].str.contains("Surprised"), 1, 0)
df

,conversation_id,message,sentiment,Curious,Happy,Sad,Neutral,Surprised
0,1,Are you a fan of Google or Microsoft?,Curious to dive deeper,1,0,0,0,0
1,1,Both are excellent technology they are helpfu...,Curious to dive deeper,1,0,0,0,0
2,1,"I'm not a huge fan of Google, but I use it a...",Curious to dive deeper,1,0,0,0,0
3,1,Google provides online related services and p...,Curious to dive deeper,1,0,0,0,0
4,1,"Yeah, their services are good. I'm just not a...",Curious to dive deeper,1,0,0,0,0
...,...,...,...,...,...,...,...,...
188373,8628,"Wow, it does not seem like that long. Since I...",Surprised,0,0,0,0,1
188374,8628,"I havent seen that episode, I might google it...",Curious to dive deeper,1,0,0,0,0
188375,8628,I don't think I have either. That's an insane...,Curious to dive deeper,1,0,0,0,0
188376,8628,"I did, my little brother used to love Thomas ...",Happy,0,1,0,0,0


In [54]:
#Number of messages in each conversation
df1 = df['conversation_id'].value_counts()
#Copy datafram to manipulate
manipulated = df.copy()


In [55]:
#labels each conversation with two distinct users
manipulated["User"] = ""
current_user = 1
for x in range(df1[1]):
    if (x % 2 == 0):
        manipulated['User'][x] = current_user
    else:
        manipulated['User'][x] = current_user+1
current_user = current_user + 2
for i in range(len(df.index)-1):
    if (df['conversation_id'][i] == df['conversation_id'][i+1]):
        pass
    else:
        for y in range(df1[df['conversation_id'][i+1]]):
            if (y % 2 == 0):
                manipulated['User'][i+1+y] = current_user
            else:
                manipulated['User'][i+1+y] = current_user+1
        current_user = current_user + 2

<ipython-input-55-e6be660453cd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manipulated['User'][x] = current_user
<ipython-input-55-e6be660453cd>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manipulated['User'][x] = current_user+1
<ipython-input-55-e6be660453cd>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manipulated['User'][i+1+y] = current_user
<ipython-input-55-e6be660453cd>:17: SettingWithCopyWarning: 
A value is try

In [57]:
manipulated = manipulated.set_index('User')
manipulated

,conversation_id,message,sentiment,Curious,Happy,Sad,Neutral,Surprised
User,,,,,,,,
1,1,Are you a fan of Google or Microsoft?,Curious to dive deeper,1,0,0,0,0
2,1,Both are excellent technology they are helpfu...,Curious to dive deeper,1,0,0,0,0
1,1,"I'm not a huge fan of Google, but I use it a...",Curious to dive deeper,1,0,0,0,0
2,1,Google provides online related services and p...,Curious to dive deeper,1,0,0,0,0
1,1,"Yeah, their services are good. I'm just not a...",Curious to dive deeper,1,0,0,0,0
...,...,...,...,...,...,...,...,...
17255,8628,"Wow, it does not seem like that long. Since I...",Surprised,0,0,0,0,1
17256,8628,"I havent seen that episode, I might google it...",Curious to dive deeper,1,0,0,0,0
17255,8628,I don't think I have either. That's an insane...,Curious to dive deeper,1,0,0,0,0
